In [18]:
import pandas as pd
import pprint

In [15]:
from functions import accounts_dict

In [16]:
# accounts dictionary
acct_dict = accounts_dict()

# components
acct_category_options = [*acct_dict.keys()]
acct_category_options.insert(0, 'Alle')

In [19]:
pprint.pprint(acct_dict)

{'Direct Materials': {'Additives': {'Kjøp Andre tilsettingsstoffer': 4031,
                                    'Kjøp Silica': 4030,
                                    'Kjøp Tilsettingsmaterialer': 4022},
                      'Aggregates': {'Kjøp Pukk': 4021,
                                     'Kjøp Sand': 4020,
                                     'Kjøp Tilslag for overflatesjikt': 4025},
                      'Cement': {'Kjøp Sement rapid': 4011,
                                 'Kjøp Sement standard': 4010},
                      'Details': {'Kjøp Innstøpingsgods': 4230,
                                  'Kjøp Innstøpte løftedetaljer': 4220,
                                  'Kjøp Montasjedeler 1': 4250,
                                  'Kjøp Stål til formbygging': 4320,
                                  'Montasjedeler til lager': 4545},
                      'Insulation': {'Brannisolering': 4636,
                                     'Kjøp Isolasjon': 4210},
                    

In [20]:
acct_category_options

['Alle', 'Direct Materials', 'Indirect Materials', 'Logistics', 'Projects']